In [259]:
import keras
%load_ext autoreload
%autoreload 2
%matplotlib inline

import tensorflow as tf

import ml_layer as mll

from keras import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, RandomZoom, RandomHeight, RandomWidth, RandomRotation, Input, GlobalAveragePooling2D
from keras.losses import categorical_crossentropy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [260]:
# Create the EfficientNetV2B0 base model
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)

# Disable training completely
base_model.trainable = False

In [261]:
# Create data augmentation stage
data_augmentation = Sequential([
    RandomZoom(.2),
    RandomHeight(.2),
    RandomWidth(.2),
    RandomRotation(.2)
])

In [262]:
# Create input layer
inputs = Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = Dense(10, activation="softmax", name="output_layer")(x)

# Create the model
model = Model(inputs, outputs)

# Compile the model
model.compile(loss=categorical_crossentropy,
                optimizer=Adam(),
                metrics=["accuracy"])

## Base-Layer

Actions to perform:

1. List all base layers
2. Set first 10 layers te be trainable and list first 20 layers
3. List first 20 layers, trainable only
4. List first 20 layers, non-trainable only
5. Set last 10 layers to be trainable and list last 20 layers
6. List all base layers and check if first 10 and last 10 are trainable

In [263]:
# 1. List all base layers
mll.list_model(model=base_model, recursive=True, include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_17                   (InputLayer            ) | False     | [(None, None, None, 3)] 
1     | rescaling_16               (Rescaling             ) | False     | (None, None, None, 3)   
2     | normalization_16           (Normalization         ) | False     | (None, None, None, 3)   
3     | stem_conv                  (Conv2D                ) | False     | (None, None, None, 32)  
4     | stem_bn                    (BatchNormalization    ) | False     | (None, None, None, 32)  
5     | stem_activation            (Activation            ) | False     | (None, None, None, 32)  
6     | block1a_project_conv       (Conv2D                ) | False     | (None, None, None, 16)  
7     | block1a_project_bn         (BatchNormalization    ) | False     | (None, None, None, 16)  
8     | block1a_project_activation (Activation            ) | False     | (None, None, None, 16)  
9     | block2a_expand

In [264]:
# 2. Set first 10 layers te be trainable and list first 10 layers
mll.set_trainable_on_first_n_layers(model=base_model, n=10, trainable=True)

layers = mll.collect_layers(model=base_model)
mll.list_layers(layers[:20], include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                     | Trainable | Output Shape
0     | input_17                   (InputLayer        ) | True      | [(None, None, None, 3)]
1     | rescaling_16               (Rescaling         ) | True      | (None, None, None, 3)  
2     | normalization_16           (Normalization     ) | True      | (None, None, None, 3)  
3     | stem_conv                  (Conv2D            ) | True      | (None, None, None, 32) 
4     | stem_bn                    (BatchNormalization) | True      | (None, None, None, 32) 
5     | stem_activation            (Activation        ) | True      | (None, None, None, 32) 
6     | block1a_project_conv       (Conv2D            ) | True      | (None, None, None, 16) 
7     | block1a_project_bn         (BatchNormalization) | True      | (None, None, None, 16) 
8     | block1a_project_activation (Activation        ) | True      | (None, None, None, 16) 
9     | block2a_expand_conv        (Conv2D            ) | True      | (

In [265]:
# 3. List first 20 layers, trainable only
mll.list_layers(layers[:20], include_trainable=True, include_non_trainable=False)

Row   | Name (Type)                                     | Trainable | Output Shape
0     | input_17                   (InputLayer        ) | True      | [(None, None, None, 3)]
1     | rescaling_16               (Rescaling         ) | True      | (None, None, None, 3)  
2     | normalization_16           (Normalization     ) | True      | (None, None, None, 3)  
3     | stem_conv                  (Conv2D            ) | True      | (None, None, None, 32) 
4     | stem_bn                    (BatchNormalization) | True      | (None, None, None, 32) 
5     | stem_activation            (Activation        ) | True      | (None, None, None, 32) 
6     | block1a_project_conv       (Conv2D            ) | True      | (None, None, None, 16) 
7     | block1a_project_bn         (BatchNormalization) | True      | (None, None, None, 16) 
8     | block1a_project_activation (Activation        ) | True      | (None, None, None, 16) 
9     | block2a_expand_conv        (Conv2D            ) | True      | (

In [266]:
# 4. List first 20 layers, non-trainable only
mll.list_layers(layers[:20], include_trainable=False, include_non_trainable=True)

Row   | Name (Type)                                     | Trainable | Output Shape
10    | block2a_expand_bn          (BatchNormalization) | False     | (None, None, None, 64) 
11    | block2a_expand_activation  (Activation        ) | False     | (None, None, None, 64) 
12    | block2a_project_conv       (Conv2D            ) | False     | (None, None, None, 32) 
13    | block2a_project_bn         (BatchNormalization) | False     | (None, None, None, 32) 
14    | block2b_expand_conv        (Conv2D            ) | False     | (None, None, None, 128)
15    | block2b_expand_bn          (BatchNormalization) | False     | (None, None, None, 128)
16    | block2b_expand_activation  (Activation        ) | False     | (None, None, None, 128)
17    | block2b_project_conv       (Conv2D            ) | False     | (None, None, None, 32) 
18    | block2b_project_bn         (BatchNormalization) | False     | (None, None, None, 32) 
19    | block2b_drop               (Dropout           ) | False     | (

In [267]:
# 5. Set last 10 layers to be trainable and list last 20 layers
mll.set_trainable_on_last_n_layers(model=base_model, n=10, trainable=True)

layers = mll.collect_layers(model=base_model)
mll.list_layers(layers[-20:], include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                        | Trainable | Output Shape
0     | block6g_drop              (Dropout               ) | False     | (None, None, None, 192) 
1     | block6g_add               (Add                   ) | False     | (None, None, None, 192) 
2     | block6h_expand_conv       (Conv2D                ) | False     | (None, None, None, 1152)
3     | block6h_expand_bn         (BatchNormalization    ) | False     | (None, None, None, 1152)
4     | block6h_expand_activation (Activation            ) | False     | (None, None, None, 1152)
5     | block6h_dwconv2           (DepthwiseConv2D       ) | False     | (None, None, None, 1152)
6     | block6h_bn                (BatchNormalization    ) | False     | (None, None, None, 1152)
7     | block6h_activation        (Activation            ) | False     | (None, None, None, 1152)
8     | block6h_se_squeeze        (GlobalAveragePooling2D) | False     | (None, 1152)            
9     | block6h_se_reshape      

In [268]:
# 6. List all base layers and check if first 10 and last 10 are trainable
mll.list_model(model=base_model, include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_17                   (InputLayer            ) | True      | [(None, None, None, 3)] 
1     | rescaling_16               (Rescaling             ) | True      | (None, None, None, 3)   
2     | normalization_16           (Normalization         ) | True      | (None, None, None, 3)   
3     | stem_conv                  (Conv2D                ) | True      | (None, None, None, 32)  
4     | stem_bn                    (BatchNormalization    ) | True      | (None, None, None, 32)  
5     | stem_activation            (Activation            ) | True      | (None, None, None, 32)  
6     | block1a_project_conv       (Conv2D                ) | True      | (None, None, None, 16)  
7     | block1a_project_bn         (BatchNormalization    ) | True      | (None, None, None, 16)  
8     | block1a_project_activation (Activation            ) | True      | (None, None, None, 16)  
9     | block2a_expand

## Complete Model

Actions to perform:

1. List all complete model layers, first 15 should be trainable and last 10
2. Disable training completely and check layers, all layers should have training disabled
3. Set first 10 layers te be trainable and list first 20 layers
4. List all first 20 layers, trainable only
5. List all first 20 layers, non-trainable only
6. Set last 10 layers to be trainable and list last 20 layers
7. List all layers and check if first 10 and last 10 are trainable
9. List all base layers and check if first 10 and last 10 are trainable
10. Save model
11. Load model
12. List all base layers and check if first 10 and last 10 are trainable

In [269]:
# 1. List all complete model layers, first 15 should be trainable and last 10
mll.list_model(model=model, recursive=True, include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_layer                (InputLayer            ) | True      | [(None, 224, 224, 3)]   
1     | random_zoom_25             (RandomZoom            ) | True      | (None, 224, 224, 3)     
2     | random_height_25           (RandomHeight          ) | True      | (None, None, 224, 3)    
3     | random_width_25            (RandomWidth           ) | True      | (None, None, None, 3)   
4     | random_rotation_25         (RandomRotation        ) | True      | (None, None, None, 3)   
5     | input_17                   (InputLayer            ) | True      | [(None, None, None, 3)] 
6     | rescaling_16               (Rescaling             ) | True      | (None, None, None, 3)   
7     | normalization_16           (Normalization         ) | True      | (None, None, None, 3)   
8     | stem_conv                  (Conv2D                ) | True      | (None, None, None, 32)  
9     | stem_bn       

In [270]:
# 2. Disable training completely and check layers, all layers should have training disabled
model.trainable = False
mll.list_model(model=model, recursive=True, include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_layer                (InputLayer            ) | False     | [(None, 224, 224, 3)]   
1     | random_zoom_25             (RandomZoom            ) | False     | (None, 224, 224, 3)     
2     | random_height_25           (RandomHeight          ) | False     | (None, None, 224, 3)    
3     | random_width_25            (RandomWidth           ) | False     | (None, None, None, 3)   
4     | random_rotation_25         (RandomRotation        ) | False     | (None, None, None, 3)   
5     | input_17                   (InputLayer            ) | False     | [(None, None, None, 3)] 
6     | rescaling_16               (Rescaling             ) | False     | (None, None, None, 3)   
7     | normalization_16           (Normalization         ) | False     | (None, None, None, 3)   
8     | stem_conv                  (Conv2D                ) | False     | (None, None, None, 32)  
9     | stem_bn       

In [271]:
# 3. Set first 10 layers te be trainable and list first 20 layers
mll.set_trainable_on_first_n_layers(model=model, n=10, trainable=True)

layers = mll.collect_layers(model=model)
mll.list_layers(layers[:20], include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                     | Trainable | Output Shape
0     | input_layer                (InputLayer        ) | True      | [(None, 224, 224, 3)]  
1     | random_zoom_25             (RandomZoom        ) | True      | (None, 224, 224, 3)    
2     | random_height_25           (RandomHeight      ) | True      | (None, None, 224, 3)   
3     | random_width_25            (RandomWidth       ) | True      | (None, None, None, 3)  
4     | random_rotation_25         (RandomRotation    ) | True      | (None, None, None, 3)  
5     | input_17                   (InputLayer        ) | True      | [(None, None, None, 3)]
6     | rescaling_16               (Rescaling         ) | True      | (None, None, None, 3)  
7     | normalization_16           (Normalization     ) | True      | (None, None, None, 3)  
8     | stem_conv                  (Conv2D            ) | True      | (None, None, None, 32) 
9     | stem_bn                    (BatchNormalization) | True      | (

In [272]:
# 4. List all first 20 layers, trainable only
mll.list_layers(layers[:20], include_trainable=True, include_non_trainable=False)

Row   | Name (Type)                                     | Trainable | Output Shape
0     | input_layer                (InputLayer        ) | True      | [(None, 224, 224, 3)]  
1     | random_zoom_25             (RandomZoom        ) | True      | (None, 224, 224, 3)    
2     | random_height_25           (RandomHeight      ) | True      | (None, None, 224, 3)   
3     | random_width_25            (RandomWidth       ) | True      | (None, None, None, 3)  
4     | random_rotation_25         (RandomRotation    ) | True      | (None, None, None, 3)  
5     | input_17                   (InputLayer        ) | True      | [(None, None, None, 3)]
6     | rescaling_16               (Rescaling         ) | True      | (None, None, None, 3)  
7     | normalization_16           (Normalization     ) | True      | (None, None, None, 3)  
8     | stem_conv                  (Conv2D            ) | True      | (None, None, None, 32) 
9     | stem_bn                    (BatchNormalization) | True      | (

In [273]:
# 5. List all first 20 layers, non-trainable only
mll.list_layers(layers[:20], include_trainable=False, include_non_trainable=True)

Row   | Name (Type)                                     | Trainable | Output Shape
10    | stem_activation            (Activation        ) | False     | (None, None, None, 32) 
11    | block1a_project_conv       (Conv2D            ) | False     | (None, None, None, 16) 
12    | block1a_project_bn         (BatchNormalization) | False     | (None, None, None, 16) 
13    | block1a_project_activation (Activation        ) | False     | (None, None, None, 16) 
14    | block2a_expand_conv        (Conv2D            ) | False     | (None, None, None, 64) 
15    | block2a_expand_bn          (BatchNormalization) | False     | (None, None, None, 64) 
16    | block2a_expand_activation  (Activation        ) | False     | (None, None, None, 64) 
17    | block2a_project_conv       (Conv2D            ) | False     | (None, None, None, 32) 
18    | block2a_project_bn         (BatchNormalization) | False     | (None, None, None, 32) 
19    | block2b_expand_conv        (Conv2D            ) | False     | (

In [274]:
# 6. Set last 10 layers to be trainable and list last 20 layers
mll.set_trainable_on_last_n_layers(model=model, n=10, trainable=True)

layers = mll.collect_layers(model=model)
mll.list_layers(layers[-20:], include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                        | Trainable | Output Shape
0     | block6h_expand_conv       (Conv2D                ) | False     | (None, None, None, 1152)
1     | block6h_expand_bn         (BatchNormalization    ) | False     | (None, None, None, 1152)
2     | block6h_expand_activation (Activation            ) | False     | (None, None, None, 1152)
3     | block6h_dwconv2           (DepthwiseConv2D       ) | False     | (None, None, None, 1152)
4     | block6h_bn                (BatchNormalization    ) | False     | (None, None, None, 1152)
5     | block6h_activation        (Activation            ) | False     | (None, None, None, 1152)
6     | block6h_se_squeeze        (GlobalAveragePooling2D) | False     | (None, 1152)            
7     | block6h_se_reshape        (Reshape               ) | False     | (None, 1, 1, 1152)      
8     | block6h_se_reduce         (Conv2D                ) | False     | (None, 1, 1, 48)        
9     | block6h_se_expand       

In [275]:
# 7. List all layers and check if first 10 and last 10 are trainable
mll.list_model(model=model, include_trainable=True, include_non_trainable=True)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_layer                (InputLayer            ) | True      | [(None, 224, 224, 3)]   
1     | random_zoom_25             (RandomZoom            ) | True      | (None, 224, 224, 3)     
2     | random_height_25           (RandomHeight          ) | True      | (None, None, 224, 3)    
3     | random_width_25            (RandomWidth           ) | True      | (None, None, None, 3)   
4     | random_rotation_25         (RandomRotation        ) | True      | (None, None, None, 3)   
5     | input_17                   (InputLayer            ) | True      | [(None, None, None, 3)] 
6     | rescaling_16               (Rescaling             ) | True      | (None, None, None, 3)   
7     | normalization_16           (Normalization         ) | True      | (None, None, None, 3)   
8     | stem_conv                  (Conv2D                ) | True      | (None, None, None, 32)  
9     | stem_bn       

In [276]:
mll.save_model_alt(model, directory="./models", name="model", format="h5")

In [281]:
# 10. Load weights
loaded_model = mll.load_model_alt(directory="./models", name="model", format="h5")

In [282]:
# 12. List all base layers and check if first 10 and last 10 are trainable
mll.list_model(model=loaded_model)

Row   | Name (Type)                                         | Trainable | Output Shape
0     | input_layer                (InputLayer            ) | True      | [(None, 224, 224, 3)]   
1     | random_zoom_25             (RandomZoom            ) | True      | (None, 224, 224, 3)     
2     | random_height_25           (RandomHeight          ) | True      | (None, None, 224, 3)    
3     | random_width_25            (RandomWidth           ) | True      | (None, None, None, 3)   
4     | random_rotation_25         (RandomRotation        ) | True      | (None, None, None, 3)   
5     | input_17                   (InputLayer            ) | True      | [(None, None, None, 3)] 
6     | rescaling_16               (Rescaling             ) | True      | (None, None, None, 3)   
7     | normalization_16           (Normalization         ) | True      | (None, None, None, 3)   
8     | stem_conv                  (Conv2D                ) | True      | (None, None, None, 32)  
9     | stem_bn       